# MLP with DRFP Features

This experiment uses DRFP (Differential Reaction Fingerprints) features instead of Spange descriptors.

Key changes:
1. DRFP features (2048-dim) instead of Spange (13-dim)
2. PCA for dimensionality reduction (try 50, 100, 200 components)
3. Combined with Arrhenius kinetics features
4. Same MLP architecture as baseline

GNN benchmark achieved MSE 0.0039 using DRFP - this is 25x better than current LB!

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.decomposition import PCA
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load DRFP features
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
print(f'DRFP shape: {DRFP_DF.shape}')
print(f'DRFP sparsity: {(DRFP_DF.values == 0).mean():.2%}')
print(f'Solvents: {list(DRFP_DF.index)}')

DRFP shape: (24, 2048)
DRFP sparsity: 97.43%
Solvents: ['Methanol', 'Ethylene Glycol [1,2-Ethanediol]', '1,1,1,3,3,3-Hexafluoropropan-2-ol', '2-Methyltetrahydrofuran [2-MeTHF]', 'Cyclohexane', 'IPA [Propan-2-ol]', 'Water.Acetonitrile', 'Acetonitrile', 'Acetonitrile.Acetic Acid', 'Diethyl Ether [Ether]', '2,2,2-Trifluoroethanol', 'Water.2,2,2-Trifluoroethanol', 'DMA [N,N-Dimethylacetamide]', 'Decanol', 'Ethanol', 'THF [Tetrahydrofuran]', 'Dihydrolevoglucosenone (Cyrene)', 'Ethyl Acetate', 'MTBE [tert-Butylmethylether]', 'Butanone [MEK]', 'tert-Butanol [2-Methylpropan-2-ol]', 'Dimethyl Carbonate', 'Methyl Propionate', 'Ethyl Lactate']


In [3]:
# Featurizer with DRFP + Arrhenius kinetics
class DRFPFeaturizer:
    def __init__(self, mixed=False, n_components=100):
        self.mixed = mixed
        self.n_components = n_components
        self.drfp_df = DRFP_DF
        self.pca = None
        self.fitted = False
        # 2 numeric + 3 kinetic + n_components DRFP
        self.feats_dim = 2 + 3 + n_components

    def fit_pca(self, solvent_names):
        """Fit PCA on training solvents only."""
        # Get unique solvents from training data
        if isinstance(solvent_names, pd.Series):
            unique_solvents = solvent_names.unique()
        else:
            unique_solvents = list(set(solvent_names))
        
        # Get DRFP features for training solvents
        train_drfp = self.drfp_df.loc[unique_solvents].values
        
        # Fit PCA
        self.pca = PCA(n_components=min(self.n_components, len(unique_solvents), train_drfp.shape[1]))
        self.pca.fit(train_drfp)
        self.fitted = True
        return self

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        # DRFP features
        if self.mixed:
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        # Apply PCA if fitted
        if self.fitted and self.pca is not None:
            X_drfp_reduced = self.pca.transform(X_drfp)
        else:
            X_drfp_reduced = X_drfp[:, :self.n_components]
        
        return torch.tensor(np.hstack([X_kinetic, X_drfp_reduced]))

print(f'DRFPFeaturizer defined')

DRFPFeaturizer defined


In [4]:
# MLP Model with DRFP features
class MLPInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=3, dropout=0.2):
        super(MLPInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class DRFPMLP(nn.Module):
    def __init__(self, data='single', n_models=5, n_components=100):
        super().__init__()
        self.data_type = data
        self.n_models = n_models
        self.n_components = n_components
        self.featurizer = DRFPFeaturizer(mixed=(data=='full'), n_components=n_components)
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=250, batch_size=32, lr=5e-4):
        # Fit PCA on training solvents
        if self.data_type == 'full':
            all_solvents = list(X_train["SOLVENT A NAME"]) + list(X_train["SOLVENT B NAME"])
        else:
            all_solvents = list(X_train["SOLVENT NAME"])
        self.featurizer.fit_pca(all_solvents)
        
        # Create features
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            # Set different seed for each model
            torch.manual_seed(42 + i)
            
            model = MLPInternal(input_dim, hidden_dims=[256, 128, 64]).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

print('DRFPMLP model defined')

DRFPMLP model defined


In [5]:
# Run cross-validation for single solvent data
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = DRFPMLP(data='single', n_models=5, n_components=100)
    model.train_model(train_X, train_Y, epochs=250)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [01:19<30:27, 79.46s/it]

  8%|▊         | 2/24 [02:37<28:48, 78.58s/it]

 12%|█▎        | 3/24 [03:51<26:49, 76.66s/it]

...

 92%|█████████▏| 22/24 [28:46<02:38, 79.20s/it]

 96%|█████████▌| 23/24 [30:05<01:19, 79.19s/it]

100%|██████████| 24/24 [31:24<00:00, 79.00s/it]

100%|██████████| 24/24 [31:24<00:00, 78.51s/it]


Single Solvent MSE: 0.019235


In [6]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = DRFPMLP(data='full', n_models=5, n_components=100)
    model.train_model(train_X, train_Y, epochs=250)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [04:32<54:35, 272.95s/it]

 15%|█▌        | 2/13 [09:01<49:35, 270.54s/it]

 23%|██▎       | 3/13 [13:45<46:03, 276.34s/it]

...

 85%|████████▍ | 11/13 [54:07<10:26, 313.42s/it]

 92%|█████████▏| 12/13 [59:34<05:17, 317.46s/it]

100%|██████████| 13/13 [1:05:04<00:00, 321.35s/it]

100%|██████████| 13/13 [1:05:04<00:00, 300.36s/it]


Full Data MSE: 0.015725


In [7]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')


=== FINAL RESULTS ===
Single Solvent MSE: 0.019235 (n=656)
Full Data MSE: 0.015725 (n=1227)
Overall MSE: 0.016948


In [8]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())

Submission saved to /home/submission/submission.csv
Submission shape: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.003518  0.003810  0.969494
1       1     0     0    1  0.005765  0.005679  0.961657
2       2     0     0    2  0.010774  0.009975  0.944357
3       3     0     0    3  0.018991  0.017565  0.919442
4       4     0     0    4  0.032913  0.031830  0.881414
